In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
C      = 400e-12           # [F]
tau    = 20e-3             # [s]
Rm     = tau / C           # [Ω]
gL     = 1 / Rm            # [S]
EL     = -70e-3            # [V]
Vth    = -54e-3            # [V]
Vreset = -65e-3            # [V]
Ie = [0, 250e-12, 500e-12] # [A]

In [ ]:
dt = 0.01e-3
tend = 0.1 + dt
t = np.r_[0 : tend : dt]
n_samples = t.size
Vm = np.zeros((len(Ie), n_samples))
for i,ie in enumerate(Ie):
    Vm[i,0] = EL + 10e-3
    for j in range(1,n_samples):
        Vm[i,j] = Vm[i,j-1] + dt / C * (gL * (EL - Vm[i,j-1]) + ie)
Vm_with_spikes = np.zeros(n_samples)
Vm_with_spikes[0] = EL + 10e-3
nice_spikes = True
for j in range(1,n_samples):
    Vm_with_spikes[j] = Vm_with_spikes[j-1] + \
        dt / C * (gL * (EL - Vm_with_spikes[j-1]) + np.max(Ie))
    if Vm_with_spikes[j] > Vth:
        if nice_spikes:
            Vm_with_spikes[j-1] = 10e-3
        Vm_with_spikes[j] = Vreset
t *= 1e3
Vm *= 1e3
Vm_with_spikes *= 1e3

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(5, 3))
cmap = plt.get_cmap('Set1', len(Ie))
with_spikes = True
for i,V in enumerate(Vm):
    if not with_spikes or Ie[i] != np.max(Ie):
        ax.plot(t, V, lw=2, color=cmap(i),
                label=r'I$_e$ = {:.0f} pA'.format(Ie[i]*1e12))
if with_spikes:
    ax.plot(t, Vm_with_spikes, lw=2, color=cmap(i),
           label=r'I$_e$ = {:.0f} pA'.format(Ie[i]*1e12))
ax.plot([0, tend * 1e3], Vth * 1e3 + np.zeros(2), 'k--', lw=2)
ax.grid(which='major', axis='both', lw=0.5, ls=':', color=[.6,.6,.6])
ax.set_xlabel('Time (ms)', fontsize=12)
ax.set_ylabel(r'V$_m$ (mV)', fontsize=12)
ax.legend(loc='upper left', frameon=False)
ax.set_ylim([-71, -44])
ax.set_xlim([-3, tend*1e3 + 3])
sns.despine()
fig.tight_layout()